# CS4487 - Group Project - Group 22 test part only

Name List:  
CHENG Ho Man 56208961, CHENG Hong Wai 56216309, CHONG Chun Yu 56225263

In [ ]:
# pytorch library installation
# pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117
# pip install matplotlib

In [ ]:
# import standard PyTorch modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

# import torchvision module to handle image manipulation
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

In [ ]:
# CNN
class Network(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3,6,4)            # 3@299*299 -> 6@296*296
        self.pool1 = nn.MaxPool2d(kernel_size=2) # 6@296*296 -> 6@148*148

        self.conv2 = nn.Conv2d(6,12,5)           # 6@148*148 -> 12@144*144    
        self.pool2 = nn.MaxPool2d(kernel_size=2) # 12@144*144 -> 12@72*72
        
        self.conv3 = nn.Conv2d(12,24,5)          # 12@72*72 -> 24@68*68
        self.pool3 = nn.MaxPool2d(kernel_size=2) # 24@68*68 -> 24@34*34

        self.fc1 = nn.Linear(24*34*34,24*34)
        self.fc2 = nn.Linear(24*34,24)
        self.fc3 = nn.Linear(24,2)
        
    def forward(self, t):
        t = self.pool1(F.relu(self.conv1(t)))
        t = self.pool2(F.relu(self.conv2(t)))
        t = self.pool3(F.relu(self.conv3(t)))
        
        t = torch.flatten(t,1)
        
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.fc3(t)
        return t

In [ ]:
# use GPU
# device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
# torch.cuda.empty_cache()

In [ ]:
# testing for TA
test_dir = "./data/test" # insert data path here

# transforms 
normalize = transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],std=[0.2023, 0.1994, 0.2010]) 
data_transform = transforms.Compose([transforms.ToTensor(),normalize])

test_data = datasets.ImageFolder(root=test_dir, 
                                  transform=data_transform, 
                                  target_transform=None)
test_loader = DataLoader(dataset=test_data, batch_size=35, shuffle=True)

model_test = Network()
model_test.load_state_dict(torch.load('model_cnn_3layer_final.ckpt',map_location=torch.device('cpu')))

model_test.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for img, labels in test_loader:

        preds = model_test(img)
        
        predicted = torch.max(preds.data, 1)[1]
        total += labels.size(0)
        correct += (predicted == labels).float().sum()           

    print('Testing accuracy: {} %'.format(100 * correct / total))
